# 1. Image Centering

There are two options for centering:

1. Via Googles Mediapipe library
A total of 468 facial landmarks are extracted. The images are aligned in the x and y directions so that, after alignment, the center of the line connecting the left and right lacrimal caruncles is located at the center of the image.

2. Via dlib (_recommended_)
A total of 68 facial landmarks are extracted. The images are aligned in the x and y directions so that, after alignment, the center of the line connecting the left and right lacrimal caruncles is located at the center of the image. For the images used, the alignment result based on the dlib68 landmarks is more convincing.

### a) Library import

In [ ]:
# Import der relevanten Libraries
import glob
import os
from pathlib import Path
import re
import shutil

import cv2
import imageio.v2 as imageio
from PIL import Image, ImageEnhance

import mediapipe as mp
from mlxtend.image import extract_face_landmarks
from mlxtend.image import EyepadAlign

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

from tqdm import tqdm

### b) Define paths

In [ ]:
# root: root directory with subfolders for aligned, cropped, ... images
root = r'C:\Users\calti\Documents\Masterarbeit\Bilder'

# orig_imgs: directory where unedited image files are located
imgs_orig = r'C:\Users\calti\Documents\Masterarbeit\Bilder\JPG'

# target: directory for aligned images
#imgs_aligned = r'C:\Users\calti\Documents\Masterarbeit\Bilder\Mediapipe'
imgs_aligned = r'C:\Users\calti\Documents\Masterarbeit\Bilder\mlxtend'

# (opt): Directory where the aligned and Photoshop Content Aware Fill processed images are located
#imgs_aligned_caf = r'C:\Users\calti\Documents\Masterarbeit\Bilder\Mediapipe CAF'
imgs_aligned_caf = r'C:\Users\calti\Documents\Masterarbeit\Bilder\mlxtend CAF'

# imgs_cropped: directory for cropped and resized images
#imgs_cropped = root+os.sep+'Cropped'
imgs_cropped = root + os.sep + imgs_aligned_caf.split(sep='\\')[-1] + 'Cropped'

# Directory for target images when using mlxtend's auto centering around nose function
output_dir = r'C:\Users\calti\Documents\Masterarbeit\Bilder\mlxtend target'

# exp_dir: Directory of the PsychoPy experiment in which the conditions file containing the paths to the images should be saved
exp = r'C:\Users\calti\Documents\Masterarbeit\PsychoPy'

### c) Helper functions

#### i) Image centering (Mediapipe Face Mesh 468)

Code is partially taken from: [Stackoverflow](https://stackoverflow.com/questions/59525640/how-to-center-the-content-object-of-a-binary-image-in-python)

In [ ]:
def center_image(image, landmark_results, lm1 = 133, lm2 = 362, lm3 = 168, suppress_output = True):
    """
    Centers an image based on the locations of specific facial landmarks detected by a landmark detection model.

    Args:

    image (ndarray): The image to be centered.
    landmark_results (Object): The landmark detection results from a face detection model.
    lm1 (int): The index of the first landmark point to be used for centering the image. Default is 133, corresponding to the left eye (around Caruncula lacrimalis).
    lm2 (int): The index of the second landmark point to be used for centering the image. Default is 362, corresponding to the right eye (around Caruncula lacrimalis).
    lm3 (int): The index of the third landmark point to be used for centering the image. Default is 168, corresponding to a point slightly above the inter-pupillary line.
    suppress_output (bool): If True, suppress the output of debugging information to the console. Default is True.

    Returns:

    centered_image (ndarray): The centered image.
    ipp_half (float): The x-coordinate of the landmark point in the centered image.
    ipp_half_2 (float): The y-coordinate of the landmark point in the centered image.
    angle (float): The angle in degrees between the line connecting the left and right eye landmarks and the horizontal axis.
    """
    
    
    # get image width, height and center coordinates
    height, width, chann = image.shape
    
    # Center of Original Input Image
    wi=(width/2)
    he=(height/2)

    # x, y, und z Koordinaten des relevanten Punktes
    # ipp1, ..., enthalten jeweils die x-, y- und z-Koordinaten der Landmark in normalisierter Einheit [0,1]
    ipp1 = landmark_results.multi_face_landmarks[0].landmark[lm1] # Auge Links (Caruncula lacrimalis)
    ipp2 = landmark_results.multi_face_landmarks[0].landmark[lm2] # Auge rechts (Caruncula lacrimalis)
    ipp3 = landmark_results.multi_face_landmarks[0].landmark[lm3] # (Punkt etwas oberhalb der Interpupillarlinie)
    
    # x und y Koordinaten in Pixeln [0,1] -> Pixel
    ipp1_x_px = ipp1.x*width
    ipp1_y_px = ipp1.y*height
    ipp2_x_px = ipp2.x*width
    ipp2_y_px = ipp2.y*height
    ipp3_y_px = ipp3.y*height
    ipp3_x_px = ipp3.x*width
    
    # Bestimmung der x-Koordinate des Mittelpunktes zwischen C. lacrimalis rechts und links
    # ipp2_x_px: x-Koordinate der C. lacrimalis rechts (landmark 362) [Pixel]
    # ipp1_x_px: x-Koordinate der C. lacrimalis links (landmark 133)  [Pixel]
    # ipp_half: x-Koordinate der C. lacrimalis links + die Hälfte der Distanz zwischen lm362 und lm133 [Pixel]
    #  - dies ist die x-Koordinate des Punktes, der später bei x = Bildbreite/2 liegen soll
    ipp_half = ipp1_x_px+(ipp2_x_px-ipp1_x_px)/2
    
    # Bestimmung der y-Koordinate des Punktes, der später bei y = Bildhöhe/2 liegen soll
    if ipp1_y_px < ipp2_y_px:
        ipp_half_2 = ipp1_y_px+(ipp2_y_px-ipp1_y_px)/2
    else:
        ipp_half_2 = ipp2_y_px+(ipp1_y_px-ipp2_y_px)/2
    
    # Bestimmung des Winkels zwischen der Linie zwischen C.l. sinister und dextra
    dX = ipp2_x_px - ipp1_x_px
    dY = ipp2_y_px - ipp1_y_px
    angle = np.degrees(np.arctan2(dY, dX))
    
    # Offset = Differenz zwischen (x,y) des Bildzentrums und (x,y) der Landmark
    #offsetX = (wi-ipp_half)
    #offsetY = (he-ipp_half_2)
    offsetX = (wi-ipp3_x_px)
    offsetY = (he-ipp_half_2)
    
    if suppress_output == False:
        msg = f'''
        EyeL x:   {ipp1_x_px}
        EyeR x:   {ipp2_x_px}
        EyeL y:   {ipp1_y_px}
        EyeR y:   {ipp2_y_px}
        IPP x:    {ipp_half}
        IPP y:    {ipp_half_2}
        Offset x: {offsetX}
        Offset y: {offsetY}
        Angle:    {angle}
        \n\n
        '''
        print(msg)
    
    # Affine matrix with Translations
    T = np.float32([[1, 0, offsetX], [0, 1, offsetY]]) 
    
    # WarpAffine
    centered_image = cv2.warpAffine(image, T, (width, height))
    
    # Return translated Image, x-Koordinate des Punktes zwischen C.l. sinistra und dextra, Winkel
    return centered_image, ipp_half, ipp_half_2, angle

#### ii) Image Centering (mlxtend, dlib68)

Code is partially taken from: [Stackoverflow](https://stackoverflow.com/questions/59525640/how-to-center-the-content-object-of-a-binary-image-in-python)

In [ ]:
def center_image_mlx(image, landmarks, lm1 = 39, lm2 = 42, suppress_output = True):
    """
    Centers an image based on the locations of specific facial landmarks detected by a landmark detection model.

    Args:
        image (numpy.ndarray): The image to center.
        landmarks (list): A list of facial landmarks, where each landmark is a
            tuple of (x, y) coordinates.
        lm1 (int): The index of the first landmark used to calculate the
            midpoint. Defaults to 39, which corresponds to the inner corner of
            the left eye.
        lm2 (int): The index of the second landmark used to calculate the
            midpoint. Defaults to 42, which corresponds to the inner corner of
            the right eye.
        suppress_output (bool): If True, suppresses the output of debugging
            information. Defaults to True.

    Returns:
    
    centered_image (ndarray): The centered image.
    ipp_half (float): The x-coordinate of the landmark point in the centered image.
    ipp_half_2 (float): The y-coordinate of the landmark point in the centered image.
    angle (float): The angle in degrees between the line connecting the left and right eye landmarks and the horizontal axis.
    """
    # get image width, height and center coordinates
    height, width, chann = image.shape
    
    # Center of Original Input Image
    wi=(width/2)
    he=(height/2)

    # x, y, und z Koordinaten des relevanten Punktes
    # ipp1, ..., enthalten jeweils die x-, y- und z-Koordinaten der Landmark in normalisierter Einheit [0,1]
    lm1_x = landmarks[lm1][0]
    lm1_y = landmarks[lm1][1]
    
    lm2_x = landmarks[lm2][0]
    lm2_y = landmarks[lm2][1]
    
    # Bestimmung der x-Koordinate des Mittelpunktes zwischen C. lacrimalis rechts und links
    # ipp2_x_px: x-Koordinate der C. lacrimalis rechts (landmark 362) [Pixel]
    # ipp1_x_px: x-Koordinate der C. lacrimalis links (landmark 133)  [Pixel]
    # ipp_half: x-Koordinate der C. lacrimalis links + die Hälfte der Distanz zwischen lm362 und lm133 [Pixel]
    #  - dies ist die x-Koordinate des Punktes, der später bei x = Bildbreite/2 liegen soll
    ipp_half = lm1_x+(lm2_x-lm1_x)/2
    
    # Bestimmung der y-Koordinate des Punktes, der später bei y = Bildhöhe/2 liegen soll
    if lm1_y < lm2_y:
        ipp_half_2 = lm1_y+(lm2_y-lm1_y)/2
    else:
        ipp_half_2 = lm2_y+(lm1_y-lm2_y)/2
    
    # Bestimmung des Winkels zwischen der Linie zwischen C.l. sinister und dextra
    dX = lm2_x - lm1_x
    dY = lm2_y - lm2_y
    angle = np.degrees(np.arctan2(dY, dX))
    
    # Offset = Differenz zwischen (x,y) des Bildzentrums und (x,y) der Landmark
    #offsetX = (wi-ipp_half)
    #offsetY = (he-ipp_half_2)
    offsetX = (wi-ipp_half)
    offsetY = (he-ipp_half_2)
    
    if suppress_output == False:
        msg = f'''
        EyeL x:   {lm1_x}
        EyeR x:   {lm2_x}
        EyeL y:   {lm1_y}
        EyeR y:   {lm2_y}
        IPP x:    {ipp_half}
        IPP y:    {ipp_half_2}
        Offset x: {offsetX}
        Offset y: {offsetY}
        Angle:    {angle}
        \n\n
        '''
        print(msg)
    
    # Affine matrix with Translations
    T = np.float32([[1, 0, offsetX], [0, 1, offsetY]]) 
    
    # WarpAffine
    centered_image = cv2.warpAffine(image, T, (width, height))
    
    # Return translated Image, x-Koordinate des Punktes zwischen C.l. sinistra und dextra, Winkel
    return centered_image, ipp_half, ipp_half_2, angle

#### iii) Bild zu Video Konvertierung

Code is partially taken from:  
[TheAILearner: openCV Img to Video](https://theailearner.com/2018/10/15/creating-video-from-images-using-opencv-python/)  
[TheAILearner: Image Resizing, wenn Input und Video Size unterschiedlich](https://theailearner.com/2018/11/15/changing-video-resolution-using-opencv-python/)

In [ ]:
def img_to_video(image_path_list: list, target_folder: str, output_name: str, size:tuple, fps=20,):
    """
    Converts a list of images to a video and saves it to the specified target folder with the given output name and file format.

    Args:
    image_path_list (list): A list of file paths of the images to be included in the video.
    target_folder (str): The path to the directory where the video file will be saved.
    output_name (str): The name of the video file to be saved.
    size (tuple): A tuple of width and height values representing the size of the output video.
    fps (int, optional): The frame rate of the output video. Default is 20 frames per second.

    Returns:
    None
    """
    
    
    img_array = []
    
    for filename in tqdm(image_path_list):
        
        # Read Image
        img = cv2.imread(filename)

        # Resize Image
        img_resized = cv2.resize(img, 
                                 size, 
                                 fx=0,
                                 fy=0, 
                                 interpolation = cv2.INTER_CUBIC)

        # Append image to list 
        img_array.append(img_resized)

    # Open video writer
    out = cv2.VideoWriter(target_folder+os.sep+output_name+'.mp4',
                              cv2.VideoWriter_fourcc(*'mp4v'), 
                              fps, 
                              size)

    # write images
    for i in tqdm(range(len(img_array))):
        out.write(img_array[i])

    
    # release video writer            
    out.release()

#### iv) Cropping und Resizing

Code is partially taken from: `cutting.py` provided by [Prof. Dr. Gernot Horstmann](https://www.uni-bielefeld.de/fakultaeten/psychologie/abteilung/arbeitseinheiten/01/people/scientificstaff/horstmann/)

In [ ]:
def crop_resize(input_folder: str, output_folder: str, output_width: int, output_height: int):
    """
    Crop and resize images from a source folder and save them in a target folder.

    Args:
        input_folder (str): The path to the folder containing the source images.
        output_folder (str): The path to the folder where the cropped and resized images will be saved.
        output_width (int): The desired width of the output images, in pixels.
        output_height (int): The desired height of the output images, in pixels.

    Returns:
        None.

    Raises:
        OSError: If the output folder cannot be created.

    The function crops and resizes images from the input folder, using a fixed aspect ratio and centering the
    cropping around the image center. The resulting images are saved in the output folder as JPEG files, with
    the same base name as the source files.

    """

    print(f'Lese Bilder aus dem Verzeichnis: {input_folder}.')
    print(f'Speichere cropped images in: {output_folder}.')
        
    
    # Checking if the output folder, imgs_aligned, exists
    # If it does not exist, it is created
    if not os.path.exists(imgs_cropped):
        os.makedirs(imgs_cropped)

    # Get a list of all file names in the image directory
    myImgList = os.listdir(input_folder)
    myImgListEncode = [x.encode('utf-8') for x in myImgList]

    # Iterate over file paths
    for thisImg in tqdm(myImgList):

        # open image
        img =Image.open(os.path.join(input_folder, thisImg))

        # Optional: Image Enhancer (i.e. brightness adjustments)
        #enhancer=ImageEnhance.Brightness(img)
        #img = enhancer.enhance(1.5)

        # Center of input image
        cenX = img.width//2
        cenY = img.height//2

        # Scaling Factor
        f=50

        # cropping of original image using scaling factor
        cropped = img.crop((
                cenX-(100*f),
                cenY-(100*.682*f),
                cenX+(100*f),
                cenY+(100*.682*f)
                ))

        # Resizing
        x_factor = y_factor = 0.45
        img=cropped.resize( (int(img.size[0]*x_factor), int(img.size[1]*y_factor)), Image.ANTIALIAS)
        img=cropped.resize( (int(img.size[0]*x_factor), int(img.size[1]*y_factor)), Resampling.LANCZOS)
        
        # Center of cropped image
        cenX = img.width//2
        cenY = img.height//2

        # cropping of original image using desired output width and height
        cropped = img.crop((
                cenX-(output_width/2),
                cenY-(output_height/2),
                cenX+(output_width/2),
                cenY+(output_height/2)
                ))

        # generate file name
        imgName = os.path.splitext(thisImg)[0]
        outName = imgName+".jpg"
        outFile = os.path.join(output_folder, outName)

        # save image to disk
        cropped.save(outFile)

### d) Centering proper

#### i) Mediapipe Library (468 landmarks)

__MediaPipe by Google__: [Github](https://github.com/google/mediapipe)

In [ ]:
#mp_drawing = mp.solutions.drawing_utils
#mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

# Checking if the output folder, imgs_aligned, exists
# If it does not exist, it is created
if os.path.exists(imgs_aligned) == False:
    os.mkdir(imgs_aligned)

# Creating a list of absolute file paths for original images in folder imgs_orig
IMAGE_FILES = glob.glob(imgs_orig+'\\*JPG')

print(f'''There are {len(IMAGE_FILES)} images in folder {imgs_orig}.\n
Target folder set to: {imgs_aligned}''')

# Initializing an empty list for angles between the left and right caruncula lacrimalis
angles = []

# Initializing an empty list for image names without file extension (looker ids)
# This list will be used to merge the angle data with experimental data
names = []

# Initializing an empty list for file paths of images where no landmarks were found
landmarks_not_found = []


with mp_face_mesh.FaceMesh(
    static_image_mode=True, # individual images, not video streams
    max_num_faces=1, # maximum number of faces (1 because only 1 face per image)
    refine_landmarks=True, # should the mesh be refined around the eye region?
    min_detection_confidence=0.5) as face_mesh:

    # iterate over file paths
    for file in tqdm(IMAGE_FILES):
        
        # read image
        image = cv2.imread(file)
        
        # Generate filename (preserve original Filename)
        img_path = Path(file)
        name_without_ext = img_path.stem
        name_with_ext = img_path.parts[-1]

        # Convert the BGR image to RGB before processing.
        # result contains 468 landmarks
        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # continue with next iteration if no landmark is found
    # keep track of images (paths) with no detected landmarks
        if not results.multi_face_landmarks:
            print(f'Keine Landmarks gefunden in: {file}')
            landmarks_not_found.append(file)
            continue
        
        # center image using default parameters
        centered_image, _, _, angle = center_image(image, results)
        
        # keep track of angle between line connecting lm1 and lm2 and x axis
        angles.append(float(angle))
        
        # keep track of file names without extension
        # This list will be used to merge the angle data with experimental data
        names.append(name_without_ext)
        
        # write aligned / centered image to disk
        cv2.imwrite(imgs_aligned+os.sep+name_with_ext , centered_image)

# create and export data frame with angles and looker id
df = pd.DataFrame({"angles":angles, 
                   "looker":names})
df["angles"] = round(df["angles"], 3)
df.to_csv(root+os.sep+'angles.csv', float_format="%.3f", index=False)

#### ii) mlxtend Library (68 Landmarks)

__mlextend__: [GitHub](https://github.com/rasbt/mlxtend)

__Code examples__ for mlxtend's `extract_face_landmarks`: [Sebastian Raschkas GitHub](https://rasbt.github.io/mlxtend/user_guide/image/extract_face_landmarks/#)

In [ ]:
# Checking if the output folder, imgs_aligned, exists
# If it does not exist, it is created
if os.path.exists(imgs_aligned) == False:
    os.mkdir(imgs_aligned)

# Creating a list of absolute file paths for original images in folder imgs_orig
imgs = glob.glob(imgs_orig + "\*.JPG")

print(f'''There are {len(imgs)} images in folder {imgs_orig}.\n
Target folder set to: {imgs_aligned}''')

# Initializing an empty list for angles between the left and right caruncula lacrimalis
angles = []

# Initializing an empty list for image names without file extension (looker ids)
lookerIDs = []


for file in tqdm(imgs):
    
    # read image
    img = cv2.imread(file)
    
    # extract/detect landmarks
    # landmark contains x- and y coordinates of the 68 extracte landmarks
    landmarks = extract_face_landmarks(img)
    
    # enter image using default parameters
    centered_image, _, _, angle = center_image_mlx(img, landmarks, suppress_output=True)
    
    # Generate filename (preserve original Filename)
    img_path = Path(file)
    name_without_ext = img_path.stem # Looker ID
    name_with_ext = img_path.parts[-1] # Filename für zentriertes Bild
    
    # keep track of angle between line connecting lm1 and lm2 and x axis
    angles.append(float(angle))

    # keep track of file names without extension
    # This list will be used to merge the angle data with experimental data
    lookerIDs.append(name_without_ext)

    # write aligned / centered image to disk
    cv2.imwrite(imgs_aligned+os.sep+name_with_ext , centered_image)

    
# create and export data frame with angles and looker id
df = pd.DataFrame({"angles":angles, 
                   "looker":lookerIDs})
df["angles"] = round(df["angles"], 3)
df.to_csv(root+os.sep+'angles.csv', float_format="%.3f", index=False)

#### iii) mlxtend Library (68 Landmarks) - Auto nose centering

This section allow the user to extract the average landmarks of a random sample drawn from all original, unedited images. All images are then transformed so that the individual landmarks match the average landmarks, with the nose located in the center of the original image.

__Centering around different landmark:__  
If you want to center around a different landmark, change the *__33__* in `displacement_vector = center - eyepad.target_landmarks_[33] # 33 = nose tip` (located under Step 3) to the desired landmark.

##### Step 1: Define target images (here: random sample from all unedited images)

Random sample of all original images will be copied to `output_dir` location and used for fitting the model

In [ ]:
# Checking if the output folder, imgs_aligned, exists
# If it does not exist, it is created
if not os.path.exists(output_dir):
    print("Output dir not found. Creating new one.")
    os.makedirs(output_dir)

# Creating a list of absolute file paths for original images in folder imgs_orig
imgs = glob.glob(imgs_orig + "\*.JPG")

# Draw random sample from all absolute file paths to the original unedited images
# These will be used to fit mlxtends EyePad Model
targets = random.sample(imgs, 20)

# Copy target images to folder
for path in targets:
    file_name = Path(path).parts[-1]
    file_name_new = output_dir+os.sep+file_name
    print(file_name_new)
    print(path)
    shutil.copyfile(path, file_name_new)

##### Step 2: Model Fitting

In [ ]:
# Get image dimensions and file extension
tmp_img = cv2.imread(targets[0])
height, width, _ = tmp_img.shape
file_ext = Path(targets[0]).suffix

# create EyepadAlign object
eyepad = EyepadAlign(verbose = 1)

# Fit model to target images
eyepad.fit_directory(target_img_dir = output_dir,
                     target_width=width, target_height=height,
                     file_extension=file_ext)

##### Step 3: Landmark extraction, image transformations

In [ ]:
# Get center of origin (original image)
center = np.array([width//2, height//2])

# Get displacement vector (center_x - landmark_x, center_y - landmark_y)
displacement_vector = center - eyepad.target_landmarks_[33] # 33 = nose tip

# displace landmarks (so that nosetip lies in center of origin)
nose_centered_landmarks = eyepad.target_landmarks_ + displacement_vector

# Create new EyepadAlign object
eyepad_cent_nose = EyepadAlign(verbose=1)

# Fit model to transformed/nose centered landmarks
eyepad_cent_nose.fit_values(target_landmarks=nose_centered_landmarks,
                             target_width=width, target_height=height);

# Target folder for auto aligned / nose centered images
imgs_aligned = r'C:\Users\calti\Documents\Masterarbeit\Bilder\mlxtend (auto)'

# Checking if the output folder, imgs_aligned, exists
# If it does not exist, it is created
if not os.path.exists(imgs_aligned):
    print("Output dir not found. Creating new one.")
    os.makedirs(imgs_aligned)

for img_path in imgs:
    # Read image
    img = cv2.imread(img_path)
    
    # Center-transform image around nose
    img_nose_centered = eyepad_cent_nose.transform(img)
    
    # Get file name of original file
    file_name = Path(img_path).parts[-1]
    
    # Write image to target folder
    cv2.imwrite(imgs_aligned+os.sep+file_name, img_nose_centered)


### e) (opt) Checking alignments by creating a video from original and aligned images

In [ ]:
# Original images
imgs_orig = glob.glob(imgs_orig+'\\*JPG')

# Resized images
imgs_aligned = glob.glob(imgs_aligned+'\\*jpg')

# Size of output video file (x, y)
size = (1920, 1080)

# Create Video of Original jpg files
img_to_video(imgs_orig,
             root,
            'Original_Images',
            size)

# Create Video of aligned jpg files
img_to_video(imgs_aligned,
             root,
            'Aligned_Images',
            size)

# 2. Image Cropping and Resizing

In [ ]:
crop_resize(imgs_aligned_caf, # folder containing aligned and content aware filled images
            imgs_cropped, # output folder for cropped images
            1280, # desired output width
            1024) # desired output height

# 3. Create Excel file with paths to experimental stimuli

### a) Data Frame with absolute and relative paths, looker ids and (corrected) gaze deviations

In [ ]:
# This code is used to create a training conditions file for the experiment.

# gets the paths of training images and experimental images
train_img_paths = glob.glob(exp + os.sep + "img_train" + os.sep + "*.jpg")
exp_img_paths = glob.glob(exp + os.sep + "img_exp" + os.sep + "*.jpg")

# Store the paths of all the images in a list
all_paths = [train_img_paths, exp_img_paths]

# create training conditions file
for i, l in enumerate(all_paths):
    
    # # Initialize lists for Looker-ID, gaze direction in °Sehwinkeln, relative image paths and corrected visual angle
    looker = []
    visAng = []
    rel_path = []
    visAng_corr = []

    
    # Iterate over image paths in corresponding list "l"
    for paths in l:

        # Split paths at each os sep, underscore, and dot
        split_parts = re.split(r'[\\/_.]', paths)

        # Add looker id to looker list
        looker.append(split_parts[-3])
        
        # Add visual angle to visAng list
        va = int(split_parts[-2])
        visAng.append(va)
        
        # Trainingsdaten (If statement checks if it is training data)
        if i == 1:
            # Korrektur des Sehwinkels
            if va == 0:
                visAng_corr.append(0)
            elif va == 1:
                visAng_corr.append(1.1)
            elif va == 2:
                visAng_corr.append(2.2)
            elif va == 3:
                visAng_corr.append(3.3)
            elif va == 4:
                visAng_corr.append(4.4)
            elif va == 5:
                visAng_corr.append(5.5)
            elif va == 6:
                visAng_corr.append(6.6)
            elif va == 7:
                visAng_corr.append(7.7)
            elif va == 8:
                visAng_corr.append(8.8)
            elif va == 9:
                visAng_corr.append(9.9)
            elif va == 10:
                visAng_corr.append(11.1)
            elif va == 11:
                visAng_corr.append(12.2)
            elif va == 12:
                visAng_corr.append(13.3)

        # Add relative path of image file to list
        rel_path.append(os.sep.join(paths.split(os.sep)[-2:]))
    
    # Create a DataFrame for training data or experimental data
    if i == 0:
        df_train = pd.DataFrame({'img_path': train_img_paths, 
                             'img_rel_path': rel_path, 
                             'looker': looker, 
                             'visAng': visAng})
    else:
        df_exp = pd.DataFrame({'img_path': exp_img_paths, 
                               'img_rel_path': rel_path, 
                               'looker': looker, 
                               'visAng': visAng,
                               'visAngCorr': visAng_corr})

### b) Export data frames as condition files (xlsx) for PsychoPy

In [ ]:
# export training condition file (df_train) as xlsx
#df_train.to_excel(exp+os.sep+"cond_training.xlsx", index=False)       

# List for visual angles used in the experiment
visAng_exp = [0.0, 2.2, 4.4, 6.6, 8.8, 13.3]

# List for visual angles not used in the experiment
visAng_irrelevant = [1.1, 3.3, 5.5, 7.7, 9.9, 11.1, 12.2]

# filter df for used stimuli
filtered_df = df_exp[df_exp.visAngCorr.isin(visAng_exp)].copy()

# filter df for unused stimuli
filtered_df_nonrelevant_stimuli = df_exp[df_exp.visAngCorr.isin(visAng_irrelevant)]

filtered_df["visAng"] = filtered_df["visAng"].astype('float64')
filtered_df["visAngCorr"] = filtered_df["visAngCorr"].astype('float64')

# write filtered df to disk
filtered_df.to_excel(exp+os.sep+"cond_exp.xlsx", index=False)

### c) Remove unused stimuli from experiment stimuli folder

As the experiment includes a resource manager component that preloads all images in the relative directories img_train and img_exp, removing the unneeded images is resource-saving.

In [ ]:
# Remove unused image files from experiment dir (PsychoPy experiment root folder)
for path in filtered_df_nonrelevant_stimuli["img_path"].tolist():
    os.remove(path)